In [1]:
import numpy as np
import pandas as pd
import sqlite3
import warnings
warnings.filterwarnings("ignore")

# Part 1. Subject selection
- Step 1. Selecting non-emergency cases only
- Step 2. Selecting patients with lab data captured betweend 0.5 and 14 day before the anesthesia and 48 hours after the surgery

In [2]:
data = pd.read_csv('https://api.vitaldb.net/cases')

###### step 1
data = data[data.emop == 0]
print('Non-em cases: ', data.caseid.nunique())


###### step 2
lab_results = pd.read_csv('https://api.vitaldb.net/labs')
lab_results = lab_results[lab_results.caseid.isin(data.caseid)]

# map the anesthesia start and operation end tiempoint
for col in ['opend', 'anestart']:
    lab_results[col] = lab_results.caseid.map(dict(zip(data.caseid, data[col])))
    
# select patients with data captured before ane or data after surgery
lab_results = lab_results[(lab_results.dt < lab_results.anestart) | (lab_results.dt > lab_results.opend)]
print('Patients with lab data captured before the anesthesia: ', lab_results.caseid.nunique())

# calculte hour different between the time of preoperative/postoperative records
# and the time when anesthetia started/operation ended
lab_results['days_preane'] = (lab_results.anestart -  lab_results.dt) / 60 / 60 / 24
lab_results['days_posop'] = (lab_results.dt - lab_results.opend) / 60 / 60 / 24
# selecting patients with records 0.5 - 14 days earlier than the anesthesia
lab_results['preane_avail'] = lab_results.days_preane.apply(lambda x: 1 if x >= 0.5 and x <= 14 else 0)
lab_results['postop_avail'] = lab_results.days_posop.apply(lambda x: 1 if x >= 2 else 0)
record_count = lab_results[['caseid', 'name', 'preane_avail']].groupby(['caseid', 'name']).sum().reset_index()
record_count.preane_avail = record_count.preane_avail.apply(lambda x: 1 if x > 0 else 0)
selected_patients = record_count[record_count.preane_avail == 1].caseid.unique().tolist()
print('Patients with lab data captured before the anesthesia and 48 hours after the operation: ', len(selected_patients))

Non-em cases:  5606
Patients with lab data captured before the anesthesia:  5049
Patients with lab data captured before the anesthesia and 48 hours after the operation:  4221


# Part 2. Defining complications
- Length of icu stay >= 2 days
- In-hospital mortality
- Post/pre Cretanine >= 1.5 times (48 hours after the operation)

In [3]:
def get_cr_diff(pre, post):
    try:
        return post / pre
    except:
        return np.nan
    
creatamine_after = lab_results[(lab_results.name == 'cr') & (lab_results.postop_avail == 1)]
creatamine_after = creatamine_after.sort_values(by=['caseid', 'name', 'days_posop']).drop_duplicates(keep='first')
data['postop_cr'] = data.caseid.map(dict(zip(creatamine_after.caseid, creatamine_after.result)))
data['pre_post_cr'] = data.apply(lambda x: get_cr_diff(x.preop_cr, x.postop_cr), axis=1) 
data['complications'] = data.apply(lambda x: 1 if any([x.death_inhosp == 1,
                                              x.icu_days >= 2,
                                              x.pre_post_cr >= 1.5]) else 0, axis=1)
print('Patients with postoperative complictions: ', data[data.complications == 1].caseid.nunique())

Patients with postoperative complictions:  304


# Part 3. Pulling lab data

In [4]:
b4 = lab_results[lab_results.caseid.isin(selected_patients)]
b4 = b4.sort_values(by=['caseid', 'name', 'days_preane'], ascending=[True, True, False])
b4 = b4.drop_duplicates(subset=['caseid', 'name'], keep='first')
b4 = b4[['caseid', 'dt', 'name', 'result', 'days_preane']]
b4.to_excel('00_preop_lab_results_extrated.xlsx', index = False)
# pivot lab data
b4 = b4[['caseid', 'name', 'result']]
b4 = b4.pivot(index=['caseid'], columns='name', values='result')
b4.columns = ['b4_' + x for x in list(b4.columns)]
b4 = b4.reset_index()
b4.to_excel('00_preop_lab_results_pivoted.xlsx', index = False)

# Part 4. Prepropcess data

In [5]:
def onehot(df, labels, variable='department'):
    cols_add = []
    labels = {x.split(': ')[1]: x.split(': ')[0]  for x in labels.split(' | ')}
    values = df[variable].map(labels).apply(lambda x: str(x).zfill(2))
    cols_add = [variable + '_' + v for v in list(labels.values())]
    dummies = pd.get_dummies(values, prefix=variable)
    return dummies, values

cols = ['caseid', 'complications', 'icu_days', 'death_inhosp', 'pre_post_cr', 'age', 'sex', 'height',
        'weight', 'bmi', 'asa', 'optype', 'approach', 'emop', 'ane_type',
        'preop_htn', 'preop_dm', 'preop_arrhythmia', 'preop_pft']
data = data[[x for x in cols if x in data.columns]]
data.asa = data.asa.fillna(7).apply(lambda x: 'cat' + str(int(float(x))).zfill(2)).astype(str)
data['sex'] = data.sex.map({'M': 0, 'F': 1}).astype(int)
variables_labels = {}
cols_to_rm = ['optype', 'approach', 'ane_type', 'preop_pft', 'asa']
for col in cols_to_rm:
    keys = list(dict(data[col].value_counts()).keys())
    label = ' | '.join(['%s: %s'%(i, keys[i]) for i in range(len(keys))])
    variables_labels[col] = label
    values, values_old = onehot(data, label, variable=col)
    data[col] = values_old
    data = pd.concat([data, values], axis=1)

variables_labels

{'optype': '0: Colorectal | 1: Biliary/Pancreas | 2: Stomach | 3: Major resection | 4: Others | 5: Minor resection | 6: Breast | 7: Transplantation | 8: Thyroid | 9: Hepatic | 10: Vascular',
 'approach': '0: Open | 1: Videoscopic | 2: Robotic',
 'ane_type': '0: General | 1: Spinal | 2: Sedationalgesia',
 'preop_pft': '0: Normal | 1: Mild obstructive | 2: Mild restrictive | 3: Moderate obstructive | 4: Mixed or pure obstructive | 5: Severe restrictive | 6: Moderate restrictive | 7: Borderline obstructive | 8: Severe obstructive',
 'asa': '0: cat02 | 1: cat01 | 2: cat03 | 3: cat07 | 4: cat06 | 5: cat04'}

In [6]:
data = data.merge(b4, on=['caseid'], how='outer')
selected_data = data[data.caseid.isin(selected_patients)]
excluded_data = data[~data.caseid.isin(selected_patients)]
selected_data.to_excel('01_data_compiled_selected_patients_4221.xlsx', index=False)
excluded_data.to_excel('01_data_compiled_excluded_patients_1385.xlsx', index=False)

print('Selected patients with postoperative complications: ',
      selected_data[selected_data.complications == 1].caseid.nunique())

Selected patients with postoperative complications:  264
